In [1]:
import pandas as pd
import numpy as np
import requests
from io import StringIO

In [2]:
orig_url = 'https://drive.google.com/file/d/1uXK-rG07T1njpXvTNxNZcmJrVJQmfeRc/view?usp=sharing'
file_id = orig_url.split('/')[-2]
dwn_url = 'https://drive.google.com/uc?export=download&id=' + file_id

purchased = pd.read_excel(dwn_url, skiprows=1, names=['Desc', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018',
                                                      'Growth ratio'])
purchased

,Desc,2011,2012,2013,2014,2015,2016,2017,2018,Growth ratio
0,Aantal (x 1000),1198,1035,1008,1051,983,928,957,1011,0.06
1,Waarde (x € 1000),879240,769159,797264,886605,899020,936829,976034,1220291,0.25


In [3]:
#Overview
print(purchased.head())
print(purchased.shape)
# purchased.columns() # It looks like there is no such method for data frames.

                Desc    2011    2012    2013    2014    2015    2016    2017  \
0    Aantal (x 1000)    1198    1035    1008    1051     983     928     957   
1  Waarde (x € 1000)  879240  769159  797264  886605  899020  936829  976034   

      2018  Growth ratio  
0     1011          0.06  
1  1220291          0.25  
(2, 10)


In [4]:
#Checking the types
purchased.dtypes

Desc             object
2011              int64
2012              int64
2013              int64
2014              int64
2015              int64
2016              int64
2017              int64
2018              int64
Growth ratio    float64
dtype: object

In [5]:
#Adding the missing column. To be able to extrapolate the missing values later
purchased['2019']=""

In [6]:
# Calculating 2019 quantity based on growth ratio
purchased['2019'] = round(purchased['2018']*(1+purchased['Growth ratio']), 0).astype(int)
purchased

,Desc,2011,2012,2013,2014,2015,2016,2017,2018,Growth ratio,2019
0,Aantal (x 1000),1198,1035,1008,1051,983,928,957,1011,0.06,1072
1,Waarde (x € 1000),879240,769159,797264,886605,899020,936829,976034,1220291,0.25,1525364


In [7]:
#Taking out the columns and the row that we won't be needing
total_purchased = purchased.drop(['2011', '2012', '2013', '2014', '2015', 'Growth ratio'],axis=1)
total_purchased = total_purchased.loc[total_purchased['Desc'] == 'Aantal (x 1000)']

In [8]:
total_purchased['Desc'] = 'Purchased bikes NL'

In [9]:
total_purchased

,Desc,2016,2017,2018,2019
0,Purchased bikes NL,928,957,1011,1072


In [10]:
#Displaying the correct unit to the Total_purchased df
total_purchased = total_purchased[['2016', '2017', '2018', '2019']].apply(lambda x: x*1000)
total_purchased.insert(0, 'Desc', 'Purchased bikes')


In [11]:
total_purchased 

,Desc,2016,2017,2018,2019
0,Purchased bikes,928000,957000,1011000,1072000


In [12]:
#Importing a separate table to be added to the already inserted 'purchase' one:

url_1 = 'https://drive.google.com/file/d/1y1nWsBv4RHFIGLqAcgH84Iqq1a2rEWW-/view?usp=sharing'
file_id = url_1.split('/')[-2]
url = 'https://drive.google.com/uc?export=download&id=' + file_id

purchased_ams = pd.read_excel(url, skiprows=1, nrows=6, names=['Desc', '2014', '2015', '2016','2017','2018','2019','tbd'])
purchased_ams 

,Desc,2014,2015,2016,2017,2018,2019,tbd
0,purchased NL x1000,1051.00000,983.000000,928.000000,957.000000,1011.000000,1071.660000,NaN
1,purchased AMS x1000,163.11487,156.000497,92.802719,89.994155,86.224719,80.257868,NaN
2,Amsterdam,11087.00000,12441.000000,10239.000000,9364.000000,8671.000000,9393.000000,NaN
3,Nederland,71437.00000,78394.000000,102387.000000,99577.000000,101669.000000,125422.000000,NaN
4,fetched out bikes_AVG NL,15000.00000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,NaN
5,fetched out bikes_AVG AMS,6000.00000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,NaN


In [13]:
# dropping unnecessary columns and rows:
purchased_ams = purchased_ams.drop(['2014','2015','tbd'], axis =1)
purchased_ams = purchased_ams.drop([0,2,3], axis =0)
purchased_ams

,Desc,2016,2017,2018,2019
1,purchased AMS x1000,92.802719,89.994155,86.224719,80.257868
4,fetched out bikes_AVG NL,15000.000000,15000.000000,15000.000000,15000.000000
5,fetched out bikes_AVG AMS,6000.000000,6000.000000,6000.000000,6000.000000


In [14]:
#changing name in row to meaningful name
purchased_ams['Desc'][1]='Purchased bikes AMS'
purchased_ams

<ipython-input-14-fd401fa397d0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  purchased_ams['Desc'][1]='Purchased bikes AMS'


,Desc,2016,2017,2018,2019
1,Purchased bikes AMS,92.802719,89.994155,86.224719,80.257868
4,fetched out bikes_AVG NL,15000.000000,15000.000000,15000.000000,15000.000000
5,fetched out bikes_AVG AMS,6000.000000,6000.000000,6000.000000,6000.000000


In [15]:
#concatenating the tables
frames = [total_purchased,purchased_ams]
total_purchased = pd.concat(frames)

In [16]:
total_purchased

,Desc,2016,2017,2018,2019
0,Purchased bikes,928000.000000,957000.000000,1.011000e+06,1.072000e+06
1,Purchased bikes AMS,92.802719,89.994155,8.622472e+01,8.025787e+01
4,fetched out bikes_AVG NL,15000.000000,15000.000000,1.500000e+04,1.500000e+04
5,fetched out bikes_AVG AMS,6000.000000,6000.000000,6.000000e+03,6.000000e+03


In [17]:
#Displaying correct unit in row 2:
#First i transpose the table, not to affect the rest of rows
total_purchased.transpose()

,0,1,4,5
Desc,Purchased bikes,Purchased bikes AMS,fetched out bikes_AVG NL,fetched out bikes_AVG AMS
2016,928000,92.8027,15000,6000
2017,957000,89.9942,15000,6000
2018,1.011e+06,86.2247,15000,6000
2019,1.072e+06,80.2579,15000,6000


In [21]:
#Applying the formula for the concerned column
total_purchased.iloc[1] = total_purchased.iloc[1].apply(lambda x: x*1000)

#Note: This actually affected the Desc. at index 1

In [23]:
total_purchased

,Desc,2016,2017,2018,2019
0,Purchased bikes,928000.0000,957000.00000,1.011000e+06,1.072000e+06
1,Purchased bikes AMSPurchased bikes AMSPurchase...,92802.7191,89994.15528,8.622472e+04,8.025787e+04
4,fetched out bikes_AVG NL,15000.0000,15000.00000,1.500000e+04,1.500000e+04
5,fetched out bikes_AVG AMS,6000.0000,6000.00000,6.000000e+03,6.000000e+03


In [26]:
#Renaming Description at row index 1
total_purchased['Desc'][1] = 'Purchased bikes AMS'

<ipython-input-26-4d554cba7cdf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_purchased['Desc'][1] = 'Purchased bikes AMS'


In [27]:
total_purchased['']

,Desc,2016,2017,2018,2019
0,Purchased bikes,928000.0000,957000.00000,1.011000e+06,1.072000e+06
1,Purchased bikes AMS,92802.7191,89994.15528,8.622472e+04,8.025787e+04
4,fetched out bikes_AVG NL,15000.0000,15000.00000,1.500000e+04,1.500000e+04
5,fetched out bikes_AVG AMS,6000.0000,6000.00000,6.000000e+03,6.000000e+03


In [32]:
convert_dict = {'Desc': object,
                '2016': int, 
                '2017': int,
                '2018': int,
                '2019': int,
               } 
  
total_purchased = total_purchased.astype(convert_dict) 

In [33]:
total_purchased

,Desc,2016,2017,2018,2019
0,Purchased bikes,928000,957000,1011000,1072000
1,Purchased bikes AMS,92802,89994,86224,80257
4,fetched out bikes_AVG NL,15000,15000,15000,15000
5,fetched out bikes_AVG AMS,6000,6000,6000,6000
